In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import warnings

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

In [3]:
import pandas as pd
import math

# to set random seed
import numpy as np

# used to create CRF model
from sklearn_crfsuite import CRF

# used to evaluate model
from sklearn_crfsuite import metrics

# for specifying f1 metrics
from sklearn.metrics import make_scorer

# for cross validation of hyperparameters
from sklearn.model_selection import RandomizedSearchCV

# to visualize the weight of parameters of the fitted model
import eli5

<IPython.core.display.Javascript object>

In [4]:
pd.set_option("max_row", 600)

<IPython.core.display.Javascript object>

In [5]:
np.random.seed(42)

<IPython.core.display.Javascript object>

In [6]:
data = pd.read_csv(
    "../1 Scraping Data and Annotating Data/tagged_data.csv", index_col=0
)

<IPython.core.display.Javascript object>

In [7]:
data.head(10)

,sentence#,word,pos,tag
0,0.0,Preheat,VB,U-Action
1,0.0,oven,NN,U-Utensil
2,0.0,to,IN,O
3,0.0,425,CD,O
4,0.0,degrees,NNS,O
5,0.0,F.,NN,O
6,1.0,Press,NN,U-Action
7,1.0,dough,NN,O
8,1.0,into,IN,O
9,1.0,the,DT,O


<IPython.core.display.Javascript object>

In [8]:
data["tag"].value_counts()

O               2498
U-Action         366
U-Ingredient     241
L-Ingredient      70
B-Ingredient      70
B-Utensil         60
U-Utensil         59
L-Utensil         59
I-Utensil         15
I-Ingredient       3
B-Action           2
L-Action           2
Name: tag, dtype: int64

<IPython.core.display.Javascript object>

In [9]:
print(len(data["word"].values))

3445


<IPython.core.display.Javascript object>

In [10]:
words = list(set(data["word"].values))
len(words)

869

<IPython.core.display.Javascript object>

In [11]:
agg_func = lambda s: [
    (w, p, t)
    for w, p, t in zip(
        s["word"].values.tolist(), s["pos"].values.tolist(), s["tag"].values.tolist()
    )
]

<IPython.core.display.Javascript object>

In [12]:
grouped = data.groupby("sentence#").apply(agg_func)

<IPython.core.display.Javascript object>

In [13]:
sentences = [s for s in grouped]

<IPython.core.display.Javascript object>

In [14]:
len(sentences)

264

<IPython.core.display.Javascript object>

In [15]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        # capture the proportion of a given label in the training set
        "bias": 1.0,
        # get the lower case form of the word
        "word.lower()": word.lower(),
        # get last 3 letters for the word
        "word[-3:]": word[-3:],
        # get last 2 letters for the word
        "word[-2:]": word[-2:],
        # check whether the word is uppercase or not
        "word.isupper()": word.isupper(),
        # check whether the word is title case or not
        "word.istitle()": word.istitle(),
        # check whether the word is digit or not, useful to identifying quantities which will be tagged as 'O'
        "word.isdigit()": word.isdigit(),
        # specifying the pos for word
        "postag": postag,
        # get first 2 letters for the POS tag
        "postag[:2]": postag[:2],
        # check if word is symbol or not
        "non_symbol_checker": word not in ["(", ")", ".", ","],
    }

    # if word is starting of sentence
    if i > 0:

        # if word is not the beginning of sentence
        # then get the word before it i.e. i-1 index
        word1 = sent[i - 1][0]

        # then get the pos before it i.e. i-1 index
        postag1 = sent[i - 1][1]

        features.update(
            {
                # setting the lower form of word at index i-1
                "-1:word.lower()": word1.lower(),
                # checking if the word at index i-1 is titlecase
                "-1:word.istitle()": word1.istitle(),
                # checking if the word at index i-1 is uppercase
                "-1:word.isupper()": word1.isupper(),
                # setting the pos of word at index i-1
                "-1:postag": postag1,
                # get first 2 letters for the POS tag for i-1 indexed word
                "-1:postag[:2]": postag1[:2],
            }
        )
    else:
        # setting the BOS or Begining of sentence to True
        features["BOS"] = True

    # if word is at the end of sentence
    if i < len(sent) - 1:

        # if word is not the end of sentence
        # then get the word after it i.e. i+1 index
        word1 = sent[i + 1][0]

        # then get the pos after it i.e. i+1 index
        postag1 = sent[i + 1][1]

        features.update(
            {
                # setting the lower form of word at index i+1
                "+1:word.lower()": word1.lower(),
                # checking if the word at index i+1 is titlecase
                "+1:word.istitle()": word1.istitle(),
                # checking if the word at index i+1 is titlecase
                "+1:word.isupper()": word1.isupper(),
                # setting the pos of word at index i+1
                "+1:postag": postag1,
                # get first 2 letters for the POS tag for i+1 indexed word
                "+1:postag[:2]": postag1[:2],
            }
        )
    else:
        # setting the EOS or End of sentence to True
        features["EOS"] = True

    return features

<IPython.core.display.Javascript object>

In [16]:
def sent2features(sent):
    """Convert sentences which are lists containing (w, p, t) into features"""
    return [word2features(sent, i) for i in range(len(sent))]

<IPython.core.display.Javascript object>

In [17]:
def sent2labels(sent):
    """Retrieve all the labels from sentences which are lists containing (w, p, t)"""
    return [label for token, postag, label in sent]

<IPython.core.display.Javascript object>

In [18]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

<IPython.core.display.Javascript object>

In [19]:
# split into train and test
boundary = math.ceil(len(X) * 0.8)

<IPython.core.display.Javascript object>

In [20]:
boundary

212

<IPython.core.display.Javascript object>

In [21]:
# train data
x_train = X[:boundary]
y_train = y[:boundary]

# test data
x_test = X[boundary:]
y_test = y[boundary:]

<IPython.core.display.Javascript object>

In [22]:
print(len(x_train))
print(len(x_test))

212
52


<IPython.core.display.Javascript object>

In [23]:
# creating CRF model with Gradient Descent
crf = CRF(algorithm="lbfgs")

<IPython.core.display.Javascript object>

In [24]:
# fitting the model using train data
crf.fit(x_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=None,
    averaging=None, c=None, c1=None, c2=None, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=None,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

<IPython.core.display.Javascript object>

In [25]:
# to get all the labels/tags of data
labels = list(crf.classes_)

<IPython.core.display.Javascript object>

In [26]:
labels

['U-Action',
 'U-Utensil',
 'O',
 'B-Utensil',
 'L-Utensil',
 'U-Ingredient',
 'B-Ingredient',
 'L-Ingredient',
 'I-Utensil',
 'I-Ingredient',
 'B-Action',
 'L-Action']

<IPython.core.display.Javascript object>

Since we are not interested in 'O' tags we will check the performance of the CRF model using f1 scores for every tag except O tags.

In [27]:
labels.remove("O")
labels

['U-Action',
 'U-Utensil',
 'B-Utensil',
 'L-Utensil',
 'U-Ingredient',
 'B-Ingredient',
 'L-Ingredient',
 'I-Utensil',
 'I-Ingredient',
 'B-Action',
 'L-Action']

<IPython.core.display.Javascript object>

In [28]:
# performing predictions based on the fitted model
y_pred = crf.predict(x_test)

<IPython.core.display.Javascript object>

In [29]:
# finding the f1 score
metrics.flat_f1_score(y_test, y_pred, average="weighted", labels=labels)

0.7169742643026694

<IPython.core.display.Javascript object>

In [30]:
# finding the f1 score
print(metrics.flat_classification_report(y_test, y_pred, labels=labels, digits=3))

              precision    recall  f1-score   support

    U-Action      0.889     0.878     0.883        82
   U-Utensil      1.000     0.353     0.522        17
   B-Utensil      1.000     0.231     0.375        13
   L-Utensil      1.000     0.308     0.471        13
U-Ingredient      0.756     0.756     0.756        45
B-Ingredient      0.538     0.583     0.560        12
L-Ingredient      0.538     0.583     0.560        12
   I-Utensil      1.000     0.333     0.500         3
I-Ingredient      0.000     0.000     0.000         0
    B-Action      0.000     0.000     0.000         0
    L-Action      0.000     0.000     0.000         0

   micro avg      0.807     0.680     0.738       197
   macro avg      0.611     0.366     0.421       197
weighted avg      0.842     0.680     0.717       197



<IPython.core.display.Javascript object>

The model overfits!!

In [31]:
eli5.show_weights(crf, top=10)

From \ To,O,B-Action,L-Action,U-Action,B-Ingredient,I-Ingredient,L-Ingredient,U-Ingredient,B-Utensil,I-Utensil,L-Utensil,U-Utensil
O,2.538,0.183,0.0,0.871,0.729,0.02,0.0,1.531,1.232,0.0,0.0,0.783
B-Action,0.0,0.0,0.921,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
L-Action,0.073,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U-Action,1.048,0.0,0.0,0.0,0.485,0.0,0.0,0.909,0.0,0.0,0.0,0.637
B-Ingredient,0.0,0.0,0.0,0.0,0.0,0.397,3.747,0.0,0.0,0.0,0.0,0.0
I-Ingredient,-0.175,0.0,0.0,0.0,0.0,0.0,0.406,0.0,0.0,0.0,0.0,0.0
L-Ingredient,0.755,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U-Ingredient,1.468,0.0,0.0,0.111,0.306,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-Utensil,-1.047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.674,2.48,0.0
I-Utensil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.54,1.829,0.0


<IPython.core.display.Javascript object>

In [32]:
y_pred[215 - 212]

['U-Action', 'O', 'O']

<IPython.core.display.Javascript object>

In [33]:
y_pred[216 - 212]

['O',
 'O',
 'O',
 'U-Ingredient',
 'O',
 'U-Action',
 'O',
 'O',
 'U-Ingredient',
 'O',
 'U-Ingredient',
 'O',
 'U-Ingredient',
 'O',
 'O',
 'O',
 'O',
 'O']

<IPython.core.display.Javascript object>

**Regularization**

In [34]:
# creating a CRF Regularized model
crf_reg = CRF(algorithm="lbfgs", c1=5, c2=0.1)

<IPython.core.display.Javascript object>

In [35]:
# fitting the hyperparameters
crf_reg.fit(x_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=None,
    averaging=None, c=None, c1=5, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=None,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

<IPython.core.display.Javascript object>

In [36]:
# predict using the best CRF model
y_pred = crf_reg.predict(x_test)

<IPython.core.display.Javascript object>

In [37]:
# print the f1 evaluation metric
print(metrics.flat_classification_report(y_test, y_pred, labels=labels, digits=3))

              precision    recall  f1-score   support

    U-Action      0.877     0.695     0.776        82
   U-Utensil      1.000     0.118     0.211        17
   B-Utensil      0.500     0.077     0.133        13
   L-Utensil      0.500     0.077     0.133        13
U-Ingredient      0.681     0.711     0.696        45
B-Ingredient      0.304     0.583     0.400        12
L-Ingredient      0.304     0.583     0.400        12
   I-Utensil      0.000     0.000     0.000         3
I-Ingredient      0.000     0.000     0.000         0
    B-Action      0.000     0.000     0.000         0
    L-Action      0.000     0.000     0.000         0

   micro avg      0.652     0.543     0.593       197
   macro avg      0.379     0.259     0.250       197
weighted avg      0.710     0.543     0.566       197



<IPython.core.display.Javascript object>

In [38]:
eli5.show_weights(crf_reg, top=10)

From \ To,O,B-Action,L-Action,U-Action,B-Ingredient,I-Ingredient,L-Ingredient,U-Ingredient,B-Utensil,I-Utensil,L-Utensil,U-Utensil
O,1.423,0.0,0.0,0.856,0.0,0.0,0.0,0.886,0.778,0.0,0.0,0.052
B-Action,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
L-Action,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U-Action,0.717,0.0,0.0,0.0,0.0,0.0,0.0,0.799,0.0,0.0,0.0,0.0
B-Ingredient,0.0,0.0,0.0,0.0,0.0,0.0,5.926,0.0,0.0,0.0,0.0,0.0
I-Ingredient,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
L-Ingredient,0.478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U-Ingredient,1.195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-Utensil,-0.708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.915,4.341,0.0
I-Utensil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.193,0.0


<IPython.core.display.Javascript object>

## Untagged Data Stuff

In [39]:
untagged_test_data = pd.read_csv(
    "../2 Testing and Annotating Testing Data/Untagged Test Data/untagged_test_data tagged.csv"
)

<IPython.core.display.Javascript object>

In [40]:
untagged_test_data["tag"].value_counts()

O               2481
U-Action         319
U-Ingredient     258
U-Utensil         63
L-Utensil         60
B-Utensil         59
B-Ingredient      51
L-Ingredient      50
I-Utensil         17
I-Ingredient       4
B-Action           1
I-Action           1
Name: tag, dtype: int64

<IPython.core.display.Javascript object>

In [41]:
untagged_test_data.head()

,recipe_name,Step#,word,pos,tag
0,homemade vegetable soup from a can,0,Combine,VB,U-Action
1,homemade vegetable soup from a can,0,all,DT,O
2,homemade vegetable soup from a can,0,ingredients,NNS,O
3,homemade vegetable soup from a can,0,in,IN,O
4,homemade vegetable soup from a can,0,large,JJ,B-Utensil


<IPython.core.display.Javascript object>

In [42]:
len(untagged_test_data)

3364

<IPython.core.display.Javascript object>

In [43]:
agg_func_test = lambda s: [
    (w, p, t)
    for w, p, t in zip(
        s["word"].values.tolist(), s["pos"].values.tolist(), s["tag"].values.tolist()
    )
]

<IPython.core.display.Javascript object>

In [44]:
grouped_test = untagged_test_data.groupby(["recipe_name", "Step#"]).apply(agg_func_test)

<IPython.core.display.Javascript object>

In [45]:
test_sentences = [s for s in grouped_test]

<IPython.core.display.Javascript object>

In [46]:
len(test_sentences)

244

<IPython.core.display.Javascript object>

In [47]:
X_untagged_test = [sent2features(s) for s in test_sentences]
y_untagged_test = [sent2labels(s) for s in test_sentences]

<IPython.core.display.Javascript object>

In [48]:
len(X_untagged_test)

244

<IPython.core.display.Javascript object>

In [49]:
pred_test = crf_reg.predict(X_untagged_test)

<IPython.core.display.Javascript object>

In [50]:
# print the f1 evaluation metric
print(
    metrics.flat_classification_report(
        y_untagged_test, pred_test, labels=labels, digits=3
    )
)

              precision    recall  f1-score   support

    U-Action      0.870     0.777     0.821       319
   U-Utensil      1.000     0.286     0.444        63
   B-Utensil      0.636     0.237     0.346        59
   L-Utensil      0.636     0.233     0.341        60
U-Ingredient      0.711     0.581     0.640       258
B-Ingredient      0.245     0.529     0.335        51
L-Ingredient      0.236     0.520     0.325        50
   I-Utensil      0.000     0.000     0.000        17
I-Ingredient      0.000     0.000     0.000         4
    B-Action      0.000     0.000     0.000         1
    L-Action      0.000     0.000     0.000         0

   micro avg      0.639     0.563     0.599       882
   macro avg      0.394     0.288     0.296       882
weighted avg      0.708     0.563     0.600       882



<IPython.core.display.Javascript object>